# Step 2: Training
This file is used for training the classification model. In this case, the model is CNN (InceptionResNetV2).

<br>

**Requirments**
- Perform preprocessing in Step1_Preprocessing.ipynb.
- Manually create the folders (i.e., InceptionResNetV2/model, InceptionResNetV2/modelWeight, and InceptionResNetV2/pic).
- Define the data paths in the Setup section.

## Setup
### Version of libraries
- Tensorflow: 2.4.0
- Keras: 2.4.0
- pandas: 1.2.1
- matplotlib 3.3.3
- numpy 1.19.5
- scikit-learn 0.24.1

In [ ]:
# Import the library for data importing and training process.
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, Conv2D, BatchNormalization ,GlobalAveragePooling2D, Input, concatenate, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, History
from keras import backend as K
from keras.applications import InceptionResNetV2

In [ ]:
# Path for Training dataset and Validation dataset.
trainDir = "/xxx/data/Train"
valDir = "/xxx/data/Valid"

#Path for saving model, modelWeight , accuracy and loss after training.
saveModel = "/xxx/InceptionResNetV2/model"
saveModelW = "/xxx/InceptionResNetV2/modelWeight"
savePic = "/xxx/InceptionResNetV2/pic"

In [ ]:
# Callback and Save Model Weights when Fit model.
class decaylr_loss(keras.callbacks.Callback):
 #For ERROR that occur when fitting the model in GCP """ AttributeError: .... no attribute '_implements_train_batch_hooks' """  
    def _implements_train_batch_hooks(self): return True
    def _implements_test_batch_hooks(self): return True
    def _implements_predict_batch_hooks(self): return True
  #####################################################################  
    def __init__(self):
        super(decaylr_loss, self).__init__()
    def on_epoch_end(self,epoch,logs={}):
        loss=logs.get('loss')
        print("loss: ",loss)
        old_lr = 0.001 #needs some adjustments
        new_lr= old_lr*np.exp(loss) #lr*exp(loss)
        print("New learning rate: ", new_lr)
        K.set_value(self.model.optimizer.lr, new_lr)
lrate = decaylr_loss()

# Checkpoint for model, if the valid accuracy is improve the model will be save at some interval,so the model or weights can be loaded later to continue the training from the state saved.
wdir = saveModelW
filepath = os.path.join(wdir ,'inception_resnetv2_weight.best.h5')
checkpoint = ModelCheckpoint(filepath , monitor='val_accuracy', verbose=1 , save_best_only=True, mode='max')

In [ ]:
# Initail Image size and batch size use for Training.
img_size = 256
batch_size = 32

In [ ]:
# Import data from Training dataset directory with ImageDataGenerator from Keras.
imgDataGen = ImageDataGenerator()

train_data = imgDataGen.flow_from_directory( trainDir, target_size = (img_size, img_size), color_mode='rgb', batch_size = batch_size, class_mode='categorical')
valid_data = imgDataGen.flow_from_directory( valDir, target_size = (img_size, img_size), color_mode='rgb', batch_size = batch_size, class_mode ='categorical', shuffle=False)

In [ ]:
# To check the classes are correct or not.
print ("Train : " , list(train_data.class_indices.keys())) 
print ("Valid : " , list(valid_data.class_indices.keys()))

In [ ]:
#To count the number of imported image.
print(train_data.batch_size , 'batches train = ' , train_data.n , 'imgs')
print(valid_data.batch_size,'batches valid = ', valid_data.n, 'imgs')


## Define CNN Model : Inception_resnet_v2


In [ ]:
n_classes = 4

# base model is pre-trained weights on ImageNet
base_model = InceptionResNetV2(
    weights='imagenet', 
    include_top=False,
    input_shape=(256,256,3)
)

x = base_model.output 

# added layers to the base model
x = AveragePooling2D(pool_size=(6, 6))(x)
x = Dropout(.2)(x)
x = Flatten()(x)

# add softmax activation
predictions = Dense(n_classes, activation='softmax')(x)    

loaded_model = Model(inputs=base_model.input, outputs=predictions)
print('Use a base model with pre-trained weights on ImageNet')

In [ ]:
#To check the model that is loaded.
loaded_model.summary()

## Training Process

In [ ]:
# Stochastic gradient descent (SGD) is an optimizer which will performs a parameter update for each training example and label.
# Learning rate (lr) is a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated.
# Momentum is a method which helps accelerate gradients vectors in the right directions, thus leading to faster converging.

In [ ]:
%%time
# Set the optimizer for model before training.
# Use 0.01 learning rate (lr) at the start and use learning rate decay method which will reduce the learning rate overtime when training and validation.
# Use 0.9 momentum, when training, it would denoise the data when training and validation.
opt = SGD(lr=.01, momentum=.9)

# Compile the model before training
# Use categorical_crossentropy for more than 2 label datasets
# Use accuracy for metrics to judge the performance when training and validation
loaded_model.compile(
    optimizer=opt,
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
# Use for step_per_epoch when training and validation.They define how many batches of samples to use in one epoch for training dataset and validation dataset.
step_size_train = train_data.n//train_data.batch_size   # steps_per_epoch
step_size_val = valid_data.n//valid_data.batch_size     # validation_steps
print('each epoch is ',step_size_train,step_size_val)

In [ ]:
%%time
# One epoch means all training dataset and validation dataset processed one times.
# History is infomation when training and validation such as training accuracy, valid accuracy which will be used after this section.
# Fit the model is the process to train and validate the model.
epochs = 100
hist = History()
loaded_model.fit(train_data,
                          validation_data = valid_data,                         
                          steps_per_epoch = step_size_train,       
                          validation_steps = step_size_val,
                          epochs = epochs,
                          verbose=1,
                          shuffle=True,
                          callbacks = [lrate , checkpoint, hist]) #earlystopper

## Visualization

In [ ]:
# plot acc and loss per epochs after training.
print(hist.history.keys())

# Plot training accuracy line.
plt.plot(hist.history['accuracy'])
# Plot validtion accuracy line.
plt.plot(hist.history['val_accuracy'])
# Title of the graph.
plt.title('model accuracy')
# Define y-axis of graph as accuracy.
plt.ylabel('accuracy')
# Define x-axis of graph as number of epoch.
plt.xlabel('epoch')
# To address train and valid legend.
plt.legend(['train', 'valid'])
# To save the graph at the specified path.
plt.savefig(os.path.join(savePic,'iInceptionResNetV2_accuracy.jpeg'), dpi=1000, bbox_inches='tight') 
# show the graph as output.
plt.show()

# Plot training loss line.
plt.plot(hist.history['loss'])
# Plot validation loss line.
plt.plot(hist.history['val_loss'])
# Title of the graph.
plt.title('model loss')
# Define y-axis as loss.
plt.ylabel('loss')
# Define x-axis as epoch.
plt.xlabel('epoch')
# To address train and valid legend.
plt.legend(['train', 'valid'])
# To save the graph at the specified path.
plt.savefig(os.path.join(savePic,'InceptionResNetV2_loss.jpeg'), dpi=1000, bbox_inches='tight') 
# show the graph as output.
plt.show()

## Save the trained model

In [ ]:
# Save model after training process to the specified path
loaded_model.save(os.path.join(saveModel,'InceptionResNetV2.h5'))